# Lab 2: Configuration Management with Ansible

The goal of this assignment is to set up a complete local network (domain name `infra.lan`, IP 172.16.0.0/16) with some typical services: a web application server (e.g. to host an intranet site), DHCP and DNS. A router will connect the LAN to the Internet. The table below lists the hosts in this network:

| Host name         | Alias  | IP             | Function             |
| :---------------- | :----- | :------------- | :------------------- |
| (physical system) |        | 172.16.0.1     | Your physical pc     |
| r001              | gw     | 172.16.255.254 | Router               |
| srv001            | ns,ns1 | 172.16.128.1   | Primary DNS          |
| srv002            | ns2    | 172.16.128.2   | Secondary DNS        |
| srv003            | dhcp   | 172.16.128.2   | DHCP server          |
| srv004            |        | 172.16.128.4   | Monitoring server    |
| srv100            | www    | 172.16.128.100 | Webserver            |
| ws0001            |        | (DHCP)         | workstation          |
| control           |        | 172.16.128.253 | Ansible control node |

## 2.1 Set up the controle node
- What is/are the IP addresses of this VM?
    - **lo:** `127.0.0.1` (localhostadres)
    - **eth0:** `10.0.2.15` (privaat IP adres toegewezen door VirtualBox voor communicatie met de hostmachine en is dynamisch (`DHCP`))
    - **eth1:** `172.16.128.253`  (binnen subnet: `172.16.0.0/16`)

- Check the VirtualBox network adapters of the VM and see if you can match the IP addresses with the VirtualBox adapter.

**NAT adapter IP:** `10.0.2.15` (biedt toegang tot het internet)

**Host-Only adapter, VirtualBox Host-Only Ethernet Adapter #9:** `172.16.128.253` (communicatie met hostmachine en andere VM's)

- Which Linux distribution are we running (command `lsb_release -a`)? Find some information about this distro!

`lsb_release -a` is not a valid command. The following command will give us the same information: `cat /etc/os-release`. The Linux distribution you are running is AlmaLinux 9.2 (Turquoise Kodkod)

- What version of the Linux kernel is installed (uname -a)?
To check the version of the Linux kernel installed, you can use the `uname -a` command. Version is `5.14.0-284.25.1.el9_2.x86_64`.
```bash
[vagrant@control ~]$ uname -a
Linux control 5.14.0-284.25.1.el9_2.x86_64 #1 SMP PREEMPT_DYNAMIC Tue Aug 8 04:07:27 EDT 2023 x86_64 x86_64 x86_64 GNU/Linux
```
- What version of Ansible is installed?
 To check the version of Ansible installed, you can use the `ansible --version` command. Version is `ansible [core 2.15.5]`

- Check the contents of the direcory `/vagrant/`.
```bash
[vagrant@control ~]$ ls /vagrant/
ansible  README.md  test         vagrant-hosts.yml
LICENSE  scripts    Vagrantfile
```


    The contents of this directory correspond with the `vmlab/` directory on your physical system. In fact, it is a shared directory, so changes you make on your physical system are immediately visible on the VM and vice versa.

You can start using the control node to execute Ansible commands. Ensure that you're in the correct directory (`/vagrant/ansible/`) before you do!

Feel free to improve the configuration of the control node to your liking. To make your changes persistent, update the Bash script found in `vmlab/scripts/control.sh`. This script is executed every time the VM is created, or when you run `vagrant provision control`. For example, you can install additional useful commands or Ansible dependencies, customize the bashrc file with command aliases or a fancy prompt, run the Ansible playbooks to configure the managed hosts, etc.

### Console Output

#### What is/are the IP address(es) of this VM?
```bash
[vagrant@control ~]$ ip -c a
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:7b:6a:36 brd ff:ff:ff:ff:ff:ff
    altname enp0s3
    inet 10.0.2.15/24 brd 10.0.2.255 scope global dynamic noprefixroute eth0
       valid_lft 85900sec preferred_lft 85900sec
    inet6 fe80::1dd7:4b0b:875f:7276/64 scope link noprefixroute
       valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:c0:c8:ef brd ff:ff:ff:ff:ff:ff
    altname enp0s8
    inet 172.16.128.253/16 brd 172.16.255.255 scope global noprefixroute eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fec0:c8ef/64 scope link
       valid_lft forever preferred_lft forever
[vagrant@control ~]$
```


```bash
[vagrant@control ~]$ cat /etc/os-release
NAME="AlmaLinux"
VERSION="9.2 (Turquoise Kodkod)"
ID="almalinux"
ID_LIKE="rhel centos fedora"
VERSION_ID="9.2"
PLATFORM_ID="platform:el9"
PRETTY_NAME="AlmaLinux 9.2 (Turquoise Kodkod)"
ANSI_COLOR="0;34"
LOGO="fedora-logo-icon"
CPE_NAME="cpe:/o:almalinux:almalinux:9::baseos"
HOME_URL="https://almalinux.org/"
DOCUMENTATION_URL="https://wiki.almalinux.org/"
BUG_REPORT_URL="https://bugs.almalinux.org/"

ALMALINUX_MANTISBT_PROJECT="AlmaLinux-9"
ALMALINUX_MANTISBT_PROJECT_VERSION="9.2"
REDHAT_SUPPORT_PRODUCT="AlmaLinux"
REDHAT_SUPPORT_PRODUCT_VERSION="9.2"
```

#### Version of Linux kernel
```bash
[vagrant@control ~]$ uname -a
Linux control 5.14.0-284.25.1.el9_2.x86_64 #1 SMP PREEMPT_DYNAMIC Tue Aug 8 04:07:27 EDT 2023 x86_64 x86_64 x86_64 GNU/Linux
```

#### Ansible version
```bash session
[vagrant@control ~]$ ansible --version
ansible [core 2.15.5]
  config file = None
  configured module search path = ['/home/vagrant/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /home/vagrant/.local/lib/python3.9/site-packages/ansible
  ansible collection location = /home/vagrant/.ansible/collections:/usr/share/ansible/collections
  executable location = /home/vagrant/.local/bin/ansible
  python version = 3.9.16 (main, May 29 2023, 00:00:00) [GCC 11.3.1 20221121 (Red Hat 11.3.1-4)] (/usr/bin/python3)
  jinja version = 3.1.2
  libyaml = True
```



#### Vagrant Up
#### Vagrant Up
```bash
PS C:\Users\JensV\Desktop\Hogent_23-24\Sem1\Infrastructure_Automation\infra-labs-23-24-JensVandeWynckel\vmlab> vagrant up
Bringing machine 'control' up with 'virtualbox' provider...
==> control: Box 'bento/almalinux-9' could not be found. Attempting to find and install...
    control: Box Provider: virtualbox
    control: Box Version: >= 0
==> control: Loading metadata for box 'bento/almalinux-9'
    control: URL: https://vagrantcloud.com/bento/almalinux-9
==> control: Adding box 'bento/almalinux-9' (v202309.08.0) for provider: virtualbox
    control: Downloading: https://vagrantcloud.com/bento/boxes/almalinux-9/versions/202309.08.0/providers/virtualbox/unknown/vagrant.box
    control:
==> control: Successfully added box 'bento/almalinux-9' (v202309.08.0) for 'virtualbox'!
==> control: Importing base box 'bento/almalinux-9'...
==> control: Matching MAC address for NAT networking...
==> control: Checking if box 'bento/almalinux-9' version '202309.08.0' is up to date...
==> control: Setting the name of the VM: vmlab_control_1697388676773_51441
==> control: Fixed port collision for 22 => 2222. Now on port 2200.
==> control: Clearing any previously set network interfaces...
==> control: Preparing network interfaces based on configuration...
    control: Adapter 1: nat
    control: Adapter 2: hostonly
==> control: Forwarding ports...
    control: 22 (guest) => 2200 (host) (adapter 1)
==> control: Running 'pre-boot' VM customizations...
==> control: Booting VM...
==> control: Waiting for machine to boot. This may take a few minutes...
    control: SSH address: 127.0.0.1:2200
    control: SSH username: vagrant
    control: SSH auth method: private key
    control:
    control: Vagrant insecure key detected. Vagrant will automatically replace
    control: this with a newly generated keypair for better security.
    control:
    control: Inserting generated public key within guest...
    control: Removing insecure key from the guest if it's present...
    control: Key inserted! Disconnecting and reconnecting using new SSH key...
==> control: Machine booted and ready!
==> control: Checking for guest additions in VM...
==> control: Setting hostname...
==> control: Configuring and enabling network interfaces...
==> control: Mounting shared folders...
    control: /vagrant => C:/Users/JensV/Desktop/Hogent_23-24/Sem1/Infrastructure_Automation/infra-labs-23-24-JensVandeWynckel/vmlab
==> control: Running provisioner: shell...
    control: Running: C:/Users/JensV/AppData/Local/Temp/vagrant-shell20231015-15628-528n5z.sh
    control: [LOG]  Starting server specific provisioning tasks on host control
    control: [LOG]  Installing Ansible and dependencies
    control: AlmaLinux 9 - AppStream                         2.7 MB/s |  10 MB     00:03
    control: AlmaLinux 9 - BaseOS                            2.9 MB/s | 4.7 MB     00:01
    control: AlmaLinux 9 - Extras                             18 kB/s |  18 kB     00:01
    control: Dependencies resolved.
    control: ================================================================================
    control:  Package               Architecture    Version            Repository       Size
    control: ================================================================================
    control: Installing:
    control:  epel-release          noarch          9-5.el9            extras           18 k
    control:
    control: Transaction Summary
    control: ================================================================================
    control: Install  1 Package
    control:
    control: Total download size: 18 k
    control: Installed size: 25 k
    control: Downloading Packages:
    control: epel-release-9-5.el9.noarch.rpm                  72 kB/s |  18 kB     00:00
    control: --------------------------------------------------------------------------------
    control: Total                                            24 kB/s |  18 kB     00:00
    control: Running transaction check
    control: Transaction check succeeded.
    control: Running transaction test
    control: Transaction test succeeded.
    control: Running transaction
    control:   Preparing        :
                         1/1
    control:   Installing       : epel-release-9-5.el9.noarch                            1/1
    control:   Running scriptlet: epel-release-9-5.el9.noarch                            1/1
    control: Many EPEL packages require the CodeReady Builder (CRB) repository.
    control: It is recommended that you run /usr/bin/crb enable to enable the CRB repository.
    control:
    control:   Verifying        : epel-release-9-5.el9.noarch                            1/1
    control:
    control: Installed:
    control:   epel-release-9-5.el9.noarch
    control:
    control: Complete!
    control: Extra Packages for Enterprise Linux 9 - x86_64  2.3 MB/s |  19 MB     00:08
    control: Last metadata expiration check: 0:00:02 ago on Sun 15 Oct 2023 04:52:39 PM UTC.
    control: Package psmisc-23.4-3.el9.x86_64 is already installed.
    control: Package python3-libselinux-3.5-1.el9.x86_64 is already installed.
    control: Package python3-libsemanage-3.5-1.el9.x86_64 is already installed.
    control: Dependencies resolved.
    control: ================================================================================
    control:  Package                  Arch     Version
           Repository   Size
    control: ================================================================================
    control: Installing:
    control:  bash-completion          noarch   1:2.11-4.el9               baseos      291 k
    control:  bats                     noarch   1.8.0-1.el9                epel         54 k
    control:  bind-utils               x86_64   32:9.16.23-11.el9_2.2      appstream   198 k
    control:  mc                       x86_64   1:4.8.26-5.el9             appstream   1.9 M
    control:  python3-PyMySQL          noarch   0.10.1-6.el9               appstream    93 k
    control:  python3-netaddr          noarch   0.8.0-5.el9                appstream   1.5 M
    control:  python3-pip              noarch   21.2.3-6.el9               appstream   1.7 M
    control:  tree                     x86_64   1.8.0-10.el9               baseos       55 k
    control:  vim-enhanced             x86_64   2:8.2.2637-20.el9_1        appstream   1.8 M
    control: Installing dependencies:
    control:  bind-libs                x86_64   32:9.16.23-11.el9_2.2      appstream   1.2 M
    control:  bind-license             noarch   32:9.16.23-11.el9_2.2      appstream    12 k
    control:  fstrm                    x86_64   0.6.1-3.el9                appstream    27 k
    control:  gpm-libs                 x86_64   1.20.7-29.el9              appstream    20 k
    control:  libmaxminddb             x86_64   1.5.2-3.el9                appstream    33 k
    control:  libpkgconf               x86_64   1.7.3-10.el9               baseos       35 k
    control:  libuv                    x86_64   1:1.42.0-1.el9             appstream   149 k
    control:  parallel                 noarch   20230822-1.el9             epel        417 k
    control:  perl-AutoLoader          noarch   5.74-480.el9               appstream    21 k
    control:  perl-B                   x86_64   1.80-480.el9               appstream   179 k
    control:  perl-Carp                noarch   1.50-460.el9               appstream    29 k
    control:  perl-Class-Struct        noarch   0.66-480.el9               appstream    22 k
    control:  perl-Data-Dumper         x86_64   2.174-462.el9              appstream    55 k
    control:  perl-Digest              noarch   1.19-4.el9                 appstream    25 k
    control:  perl-Digest-MD5          x86_64   2.58-4.el9                 appstream    36 k
    control:  perl-Encode              x86_64   4:3.08-462.el9             appstream   1.7 M
    control:  perl-Errno               x86_64   1.30-480.el9               appstream    15 k
    control:  perl-Exporter            noarch   5.74-461.el9               appstream    31 k
    control:  perl-Fcntl               x86_64   1.13-480.el9               appstream    20 k
    control:  perl-File-Basename       noarch   2.85-480.el9               appstream    17 k
    control:  perl-File-Path           noarch   2.18-4.el9                 appstream    35 k
    control:  perl-File-Temp           noarch   1:0.231.100-4.el9          appstream    59 k
    control:  perl-File-stat           noarch   1.09-480.el9               appstream    17 k
    control:  perl-FileHandle          noarch   2.03-480.el9               appstream    16 k
    control:  perl-Getopt-Long         noarch   1:2.52-4.el9               appstream    59 k
    control:  perl-Getopt-Std          noarch   1.12-480.el9               appstream    16 k
    control:  perl-HTTP-Tiny           noarch   0.076-460.el9              appstream    54 k
    control:  perl-IO                  x86_64   1.43-480.el9               appstream    86 k
    control:  perl-IO-Socket-IP        noarch   0.41-5.el9                 appstream    42 k
    control:  perl-IPC-Open3           noarch   1.21-480.el9               appstream    23 k
    control:  perl-MIME-Base64         x86_64   3.16-4.el9                 appstream    30 k
    control:  perl-Net-SSLeay          x86_64   1.92-2.el9                 appstream   364 k
    control:  perl-POSIX               x86_64   1.94-480.el9               appstream    96 k
    control:  perl-PathTools           x86_64   3.78-461.el9               appstream    85 k
    control:  perl-Pod-Escapes         noarch   1:1.07-460.el9             appstream    20 k
    control:  perl-Pod-Perldoc         noarch   3.28.01-461.el9            appstream    83 k
    control:  perl-Pod-Simple          noarch   1:3.42-4.el9               appstream   215 k
    control:  perl-Pod-Usage           noarch   4:2.01-4.el9               appstream    40 k
    control:  perl-Scalar-List-Utils   x86_64   4:1.56-461.el9             appstream    71 k
    control:  perl-SelectSaver         noarch   1.02-480.el9               appstream    12 k
    control:  perl-Socket              x86_64   4:2.031-4.el9              appstream    54 k
    control:  perl-Storable            x86_64   1:3.21-460.el9             appstream    95 k
    control:  perl-Symbol              noarch   1.08-480.el9               appstream    14 k
    control:  perl-Term-ANSIColor      noarch   5.01-461.el9               appstream    48 k
    control:  perl-Term-Cap            noarch   1.17-460.el9               appstream    22 k
    control:  perl-Text-ParseWords     noarch   3.30-460.el9               appstream    16 k
    control:  perl-Text-Tabs+Wrap      noarch   2013.0523-460.el9          appstream    22 k
    control:  perl-Thread-Queue        noarch   3.14-460.el9               appstream    21 k
    control:  perl-Time-Local          noarch   2:1.300-7.el9              appstream    33 k
    control:  perl-URI                 noarch   5.09-3.el9                 appstream   108 k
    control:  perl-base                noarch   2.27-480.el9               appstream    16 k
    control:  perl-constant            noarch   1.33-461.el9               appstream    23 k
    control:  perl-if                  noarch   0.60.800-480.el9           appstream    14 k
    control:  perl-interpreter         x86_64   4:5.32.1-480.el9           appstream    71 k
    control:  perl-libnet              noarch   3.13-4.el9                 appstream   125 k
    control:  perl-libs                x86_64   4:5.32.1-480.el9           appstream   2.0 M
    control:  perl-mro                 x86_64   1.23-480.el9               appstream    28 k
    control:  perl-overload            noarch   1.31-480.el9               appstream    46 k
    control:  perl-overloading         noarch   0.02-480.el9               appstream    13 k
    control:  perl-parent              noarch   1:0.238-460.el9            appstream    14 k
    control:  perl-podlators           noarch   1:4.14-460.el9             appstream   111 k
    control:  perl-subs                noarch   1.03-480.el9               appstream    12 k
    control:  perl-threads             x86_64   1:2.25-460.el9             appstream    57 k
    control:  perl-threads-shared      x86_64   1.61-460.el9               appstream    44 k
    control:  perl-vars                noarch   1.05-480.el9               appstream    13 k
    control:  pkgconf                  x86_64   1.7.3-10.el9               baseos       40 k
    control:  pkgconf-m4               noarch   1.7.3-10.el9               baseos       14 k
    control:  pkgconf-pkg-config       x86_64   1.7.3-10.el9               baseos      9.9 k
    control:  protobuf-c               x86_64   1.3.3-12.el9               baseos       35 k
    control:  python3-cffi             x86_64   1.14.5-5.el9               baseos      241 k
    control:  python3-cryptography     x86_64   36.0.1-2.el9               baseos      1.1 M
    control:  python3-ply              noarch   3.11-14.el9                baseos      103 k
    control:  python3-pycparser        noarch   2.20-6.el9                 baseos      124 k
    control:  vim-common               x86_64   2:8.2.2637-20.el9_1        appstream   6.6 M
    control:  vim-filesystem           noarch   2:8.2.2637-20.el9_1        baseos       14 k
    control: Installing weak dependencies:
    control:  perl-IO-Socket-SSL       noarch   2.073-1.el9                appstream   216 k
    control:  perl-Mozilla-CA          noarch   20200520-6.el9             appstream    12 k
    control:  perl-NDBM_File           x86_64   1.15-480.el9               appstream    22 k
    control:
    control: Transaction Summary
    control: ================================================================================
    control: Install  87 Packages
    control:
    control: Total download size: 25 M
    control: Installed size: 99 M
    control: Downloading Packages:
    control: (1/87): bind-license-9.16.23-11.el9_2.2.noarch.  47 kB/s |  12 kB     00:00
    control: (2/87): bind-utils-9.16.23-11.el9_2.2.x86_64.rp 598 kB/s | 198 kB     00:00
    control: (3/87): fstrm-0.6.1-3.el9.x86_64.rpm            202 kB/s |  27 kB     00:00
    control: (4/87): bind-libs-9.16.23-11.el9_2.2.x86_64.rpm 1.8 MB/s | 1.2 MB     00:00
    control: (5/87): gpm-libs-1.20.7-29.el9.x86_64.rpm        59 kB/s |  20 kB     00:00
    control: (6/87): libmaxminddb-1.5.2-3.el9.x86_64.rpm     112 kB/s |  33 kB     00:00
    control: (7/87): libuv-1.42.0-1.el9.x86_64.rpm           2.1 MB/s | 149 kB     00:00
    control: (8/87): perl-AutoLoader-5.74-480.el9.noarch.rpm  78 kB/s |  21 kB     00:00
    control: (9/87): mc-4.8.26-5.el9.x86_64.rpm              5.8 MB/s | 1.9 MB     00:00
    control: (10/87): perl-B-1.80-480.el9.x86_64.rpm         653 kB/s | 179 kB     00:00
    control: (11/87): perl-Carp-1.50-460.el9.noarch.rpm      377 kB/s |  29 kB     00:00
    control: (12/87): perl-Class-Struct-0.66-480.el9.noarch. 429 kB/s |  22 kB     00:00
    control: (13/87): perl-Data-Dumper-2.174-462.el9.x86_64. 933 kB/s |  55 kB     00:00
    control: (14/87): perl-Digest-1.19-4.el9.noarch.rpm      464 kB/s |  25 kB     00:00
    control: (15/87): perl-Digest-MD5-2.58-4.el9.x86_64.rpm  659 kB/s |  36 kB     00:00
    control: (16/87): perl-Errno-1.30-480.el9.x86_64.rpm     333 kB/s |  15 kB     00:00
    control: (17/87): perl-Encode-3.08-462.el9.x86_64.rpm    5.3 MB/s | 1.7 MB     00:00
    control: (18/87): perl-Exporter-5.74-461.el9.noarch.rpm  110 kB/s |  31 kB     00:00
    control: (19/87): perl-Fcntl-1.13-480.el9.x86_64.rpm      76 kB/s |  20 kB     00:00
    control: (20/87): perl-File-Basename-2.85-480.el9.noarch 304 kB/s |  17 kB     00:00
    control: (21/87): perl-File-Path-2.18-4.el9.noarch.rpm   516 kB/s |  35 kB     00:00
    control: (22/87): perl-File-Temp-0.231.100-4.el9.noarch. 809 kB/s |  59 kB     00:00
    control: (23/87): perl-File-stat-1.09-480.el9.noarch.rpm 123 kB/s |  17 kB     00:00
    control: (24/87): perl-FileHandle-2.03-480.el9.noarch.rp 109 kB/s |  16 kB     00:00
    control: (25/87): perl-Getopt-Long-2.52-4.el9.noarch.rpm 399 kB/s |  59 kB     00:00
    control: (26/87): perl-Getopt-Std-1.12-480.el9.noarch.rp 272 kB/s |  16 kB     00:00
    control: (27/87): perl-HTTP-Tiny-0.076-460.el9.noarch.rp 836 kB/s |  54 kB     00:00
    control: (28/87): perl-IO-1.43-480.el9.x86_64.rpm        1.2 MB/s |  86 kB     00:00
    control: (29/87): perl-IO-Socket-IP-0.41-5.el9.noarch.rp 660 kB/s |  42 kB     00:00
    control: (30/87): perl-IO-Socket-SSL-2.073-1.el9.noarch. 2.3 MB/s | 216 kB     00:00
    control: (31/87): perl-IPC-Open3-1.21-480.el9.noarch.rpm 260 kB/s |  23 kB     00:00
    control: (32/87): perl-MIME-Base64-3.16-4.el9.x86_64.rpm 461 kB/s |  30 kB     00:00
    control: (33/87): perl-Mozilla-CA-20200520-6.el9.noarch. 251 kB/s |  12 kB     00:00
    control: (34/87): perl-NDBM_File-1.15-480.el9.x86_64.rpm 406 kB/s |  22 kB     00:00
    control: (35/87): perl-Net-SSLeay-1.92-2.el9.x86_64.rpm  3.6 MB/s | 364 kB     00:00
    control: (36/87): perl-POSIX-1.94-480.el9.x86_64.rpm     1.1 MB/s |  96 kB     00:00
    control: (37/87): perl-PathTools-3.78-461.el9.x86_64.rpm 1.0 MB/s |  85 kB     00:00
    control: (38/87): perl-Pod-Escapes-1.07-460.el9.noarch.r 289 kB/s |  20 kB     00:00
    control: (39/87): perl-Pod-Perldoc-3.28.01-461.el9.noarc 943 kB/s |  83 kB     00:00
    control: (40/87): perl-Pod-Simple-3.42-4.el9.noarch.rpm  2.0 MB/s | 215 kB     00:00
    control: (41/87): perl-Pod-Usage-2.01-4.el9.noarch.rpm   507 kB/s |  40 kB     00:00
    control: (42/87): perl-Scalar-List-Utils-1.56-461.el9.x8 1.1 MB/s |  71 kB     00:00
    control: (43/87): perl-SelectSaver-1.02-480.el9.noarch.r 224 kB/s |  12 kB     00:00
    control: (44/87): perl-Socket-2.031-4.el9.x86_64.rpm     935 kB/s |  54 kB     00:00
    control: (45/87): perl-Storable-3.21-460.el9.x86_64.rpm  1.5 MB/s |  95 kB     00:00
    control: (46/87): perl-Symbol-1.08-480.el9.noarch.rpm    229 kB/s |  14 kB     00:00
    control: (47/87): perl-Term-ANSIColor-5.01-461.el9.noarc 744 kB/s |  48 kB     00:00
    control: (48/87): perl-Term-Cap-1.17-460.el9.noarch.rpm  398 kB/s |  22 kB     00:00
    control: (49/87): perl-Text-ParseWords-3.30-460.el9.noar 308 kB/s |  16 kB     00:00
    control: (50/87): perl-Text-Tabs+Wrap-2013.0523-460.el9. 385 kB/s |  22 kB     00:00
    control: (51/87): perl-Thread-Queue-3.14-460.el9.noarch. 336 kB/s |  21 kB     00:00
    control: (52/87): perl-Time-Local-1.300-7.el9.noarch.rpm 544 kB/s |  33 kB     00:00
    control: (53/87): perl-URI-5.09-3.el9.noarch.rpm         1.4 MB/s | 108 kB     00:00
    control: (54/87): perl-base-2.27-480.el9.noarch.rpm      269 kB/s |  16 kB     00:00
    control: (55/87): perl-constant-1.33-461.el9.noarch.rpm  375 kB/s |  23 kB     00:00
    control: (56/87): perl-if-0.60.800-480.el9.noarch.rpm    289 kB/s |  14 kB     00:00
    control: (57/87): perl-interpreter-5.32.1-480.el9.x86_64 1.1 MB/s |  71 kB     00:00
    control: (58/87): perl-libnet-3.13-4.el9.noarch.rpm      1.8 MB/s | 125 kB     00:00
    control: (59/87): perl-libs-5.32.1-480.el9.x86_64.rpm    4.1 MB/s | 2.0 MB     00:00
    control: (60/87): perl-mro-1.23-480.el9.x86_64.rpm        61 kB/s |  28 kB     00:00
    control: (61/87): perl-overload-1.31-480.el9.noarch.rpm   99 kB/s |  46 kB     00:00
    control: (62/87): perl-overloading-0.02-480.el9.noarch.r 184 kB/s |  13 kB     00:00
    control: (63/87): perl-podlators-4.14-460.el9.noarch.rpm 1.3 MB/s | 111 kB     00:00
    control: (64/87): perl-parent-0.238-460.el9.noarch.rpm   161 kB/s |  14 kB     00:00
    control: (65/87): perl-subs-1.03-480.el9.noarch.rpm      203 kB/s |  12 kB     00:00
    control: (66/87): perl-threads-2.25-460.el9.x86_64.rpm   1.1 MB/s |  57 kB     00:00
    control: (67/87): perl-threads-shared-1.61-460.el9.x86_6 849 kB/s |  44 kB     00:00
    control: (68/87): perl-vars-1.05-480.el9.noarch.rpm      262 kB/s |  13 kB     00:00
    control: (69/87): python3-netaddr-0.8.0-5.el9.noarch.rpm 3.9 MB/s | 1.5 MB     00:00
    control: (70/87): python3-PyMySQL-0.10.1-6.el9.noarch.rp 206 kB/s |  93 kB     00:00
    control: (71/87): python3-pip-21.2.3-6.el9.noarch.rpm    2.2 MB/s | 1.7 MB     00:00
    control: (72/87): vim-common-8.2.2637-20.el9_1.x86_64.rp 3.9 MB/s | 6.6 MB     00:01
    control: (73/87): bash-completion-2.11-4.el9.noarch.rpm  229 kB/s | 291 kB     00:01
    control: (74/87): vim-enhanced-8.2.2637-20.el9_1.x86_64. 1.1 MB/s | 1.8 MB     00:01
    control: (75/87): libpkgconf-1.7.3-10.el9.x86_64.rpm     528 kB/s |  35 kB     00:00
    control: (76/87): pkgconf-m4-1.7.3-10.el9.noarch.rpm     257 kB/s |  14 kB     00:00
    control: (77/87): pkgconf-1.7.3-10.el9.x86_64.rpm        408 kB/s |  40 kB     00:00
    control: (78/87): pkgconf-pkg-config-1.7.3-10.el9.x86_64 146 kB/s | 9.9 kB     00:00
    control: (79/87): protobuf-c-1.3.3-12.el9.x86_64.rpm     484 kB/s |  35 kB     00:00
    control: (80/87): python3-cffi-1.14.5-5.el9.x86_64.rpm   3.1 MB/s | 241 kB     00:00
    control: (81/87): python3-cryptography-36.0.1-2.el9.x86_ 4.0 MB/s | 1.1 MB     00:00
    control: (82/87): python3-ply-3.11-14.el9.noarch.rpm     352 kB/s | 103 kB     00:00
    control: (83/87): python3-pycparser-2.20-6.el9.noarch.rp 467 kB/s | 124 kB     00:00
    control: (84/87): vim-filesystem-8.2.2637-20.el9_1.noarc 139 kB/s |  14 kB     00:00
    control: (85/87): tree-1.8.0-10.el9.x86_64.rpm           451 kB/s |  55 kB     00:00
    control: (86/87): bats-1.8.0-1.el9.noarch.rpm            135 kB/s |  54 kB     00:00
    control: (87/87): parallel-20230822-1.el9.noarch.rpm     886 kB/s | 417 kB     00:00
    control: --------------------------------------------------------------------------------
    control: Total                                           3.2 MB/s |  25 MB     00:07
    control: Extra Packages for Enterprise Linux 9 - x86_64  1.6 MB/s | 1.6 kB     00:00
    control: Importing GPG key 0x3228467C:
    control:  Userid     : "Fedora (epel9) <epel@fedoraproject.org>"
    control:  Fingerprint: FF8A D134 4597 106E CE81 3B91 8A38 72BF 3228 467C
    control:  From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-9
    control: Key imported successfully
    control: Running transaction check
    control: Transaction check succeeded.
    control: Running transaction test
    control: Transaction test succeeded.
    control: Running transaction
    control:   Preparing        :
                         1/1
    control:   Installing       : protobuf-c-1.3.3-12.el9.x86_64                        1/87
    control:   Installing       : gpm-libs-1.20.7-29.el9.x86_64                         2/87
    control:   Installing       : perl-Digest-1.19-4.el9.noarch                         3/87
    control:   Installing       : perl-Digest-MD5-2.58-4.el9.x86_64                     4/87
    control:   Installing       : perl-B-1.80-480.el9.x86_64                            5/87
    control:   Installing       : perl-FileHandle-2.03-480.el9.noarch                   6/87
    control:   Installing       : perl-Data-Dumper-2.174-462.el9.x86_64                 7/87
    control:   Installing       : perl-libnet-3.13-4.el9.noarch                         8/87
    control:   Installing       : perl-AutoLoader-5.74-480.el9.noarch                   9/87
    control:   Installing       : perl-base-2.27-480.el9.noarch                        10/87
    control:   Installing       : perl-URI-5.09-3.el9.noarch                           11/87
    control:   Installing       : perl-Net-SSLeay-1.92-2.el9.x86_64                    12/87
    control:   Installing       : perl-Mozilla-CA-20200520-6.el9.noarch                13/87
    control:   Installing       : perl-if-0.60.800-480.el9.noarch                      14/87
    control:   Installing       : perl-IO-Socket-IP-0.41-5.el9.noarch                  15/87
    control:   Installing       : perl-Time-Local-2:1.300-7.el9.noarch                 16/87
    control:   Installing       : perl-File-Path-2.18-4.el9.noarch                     17/87
    control:   Installing       : perl-Pod-Escapes-1:1.07-460.el9.noarch               18/87
    control:   Installing       : perl-Text-Tabs+Wrap-2013.0523-460.el9.noarch         19/87
    control:   Installing       : perl-IO-Socket-SSL-2.073-1.el9.noarch                20/87
    control:   Installing       : perl-Class-Struct-0.66-480.el9.noarch                21/87
    control:   Installing       : perl-POSIX-1.94-480.el9.x86_64                       22/87
    control:   Installing       : perl-Term-ANSIColor-5.01-461.el9.noarch              23/87
    control:   Installing       : perl-IPC-Open3-1.21-480.el9.noarch                   24/87
    control:   Installing       : perl-subs-1.03-480.el9.noarch                        25/87
    control:   Installing       : perl-File-Temp-1:0.231.100-4.el9.noarch              26/87
    control:   Installing       : perl-Term-Cap-1.17-460.el9.noarch                    27/87
    control:   Installing       : perl-Pod-Simple-1:3.42-4.el9.noarch                  28/87
    control:   Installing       : perl-HTTP-Tiny-0.076-460.el9.noarch                  29/87
    control:   Installing       : perl-Socket-4:2.031-4.el9.x86_64                     30/87
    control:   Installing       : perl-SelectSaver-1.02-480.el9.noarch                 31/87
    control:   Installing       : perl-Symbol-1.08-480.el9.noarch                      32/87
    control:   Installing       : perl-File-stat-1.09-480.el9.noarch                   33/87
    control:   Installing       : perl-podlators-1:4.14-460.el9.noarch                 34/87
    control:   Installing       : perl-Pod-Perldoc-3.28.01-461.el9.noarch              35/87
    control:   Installing       : perl-Fcntl-1.13-480.el9.x86_64                       36/87
    control:   Installing       : perl-Text-ParseWords-3.30-460.el9.noarch             37/87
    control:   Installing       : perl-mro-1.23-480.el9.x86_64                         38/87
    control:   Installing       : perl-IO-1.43-480.el9.x86_64                          39/87
    control:   Installing       : perl-overloading-0.02-480.el9.noarch                 40/87
    control:   Installing       : perl-Pod-Usage-4:2.01-4.el9.noarch                   41/87
    control:   Installing       : perl-Errno-1.30-480.el9.x86_64                       42/87
    control:   Installing       : perl-File-Basename-2.85-480.el9.noarch               43/87
    control:   Installing       : perl-Getopt-Std-1.12-480.el9.noarch                  44/87
    control:   Installing       : perl-MIME-Base64-3.16-4.el9.x86_64                   45/87
    control:   Installing       : perl-Scalar-List-Utils-4:1.56-461.el9.x86_64         46/87
    control:   Installing       : perl-constant-1.33-461.el9.noarch                    47/87
    control:   Installing       : perl-Storable-1:3.21-460.el9.x86_64                  48/87
    control:   Installing       : perl-overload-1.31-480.el9.noarch                    49/87
    control:   Installing       : perl-parent-1:0.238-460.el9.noarch                   50/87
    control:   Installing       : perl-vars-1.05-480.el9.noarch                        51/87
    control:   Installing       : perl-Getopt-Long-1:2.52-4.el9.noarch                 52/87
    control:   Installing       : perl-Carp-1.50-460.el9.noarch                        53/87
    control:   Installing       : perl-Exporter-5.74-461.el9.noarch                    54/87
    control:   Installing       : perl-NDBM_File-1.15-480.el9.x86_64                   55/87
    control:   Installing       : perl-PathTools-3.78-461.el9.x86_64                   56/87
    control:   Installing       : perl-Encode-4:3.08-462.el9.x86_64                    57/87
    control:   Installing       : perl-libs-4:5.32.1-480.el9.x86_64                    58/87
    control:   Installing       : perl-interpreter-4:5.32.1-480.el9.x86_64             59/87
    control:   Installing       : perl-threads-1:2.25-460.el9.x86_64                   60/87
    control:   Installing       : perl-threads-shared-1.61-460.el9.x86_64              61/87
    control:   Installing       : perl-Thread-Queue-3.14-460.el9.noarch                62/87
    control:   Installing       : parallel-20230822-1.el9.noarch                       63/87
    control:   Installing       : vim-filesystem-2:8.2.2637-20.el9_1.noarch            64/87
    control:   Installing       : vim-common-2:8.2.2637-20.el9_1.x86_64                65/87
    control:   Installing       : python3-ply-3.11-14.el9.noarch                       66/87
    control:   Installing       : python3-pycparser-2.20-6.el9.noarch                  67/87
    control:   Installing       : python3-cffi-1.14.5-5.el9.x86_64                     68/87
    control:   Installing       : python3-cryptography-36.0.1-2.el9.x86_64             69/87
    control:   Installing       : pkgconf-m4-1.7.3-10.el9.noarch                       70/87
    control:   Installing       : libpkgconf-1.7.3-10.el9.x86_64                       71/87
    control:   Installing       : pkgconf-1.7.3-10.el9.x86_64                          72/87
    control:   Installing       : pkgconf-pkg-config-1.7.3-10.el9.x86_64               73/87
    control:   Installing       : libuv-1:1.42.0-1.el9.x86_64                          74/87
    control:   Installing       : libmaxminddb-1.5.2-3.el9.x86_64                      75/87
    control:   Installing       : fstrm-0.6.1-3.el9.x86_64                             76/87
    control:   Installing       : bind-license-32:9.16.23-11.el9_2.2.noarch            77/87
    control:   Installing       : bind-libs-32:9.16.23-11.el9_2.2.x86_64               78/87
    control:   Installing       : bind-utils-32:9.16.23-11.el9_2.2.x86_64              79/87
    control:   Installing       : bash-completion-1:2.11-4.el9.noarch                  80/87
    control:   Installing       : python3-PyMySQL-0.10.1-6.el9.noarch                  81/87
    control:   Installing       : vim-enhanced-2:8.2.2637-20.el9_1.x86_64              82/87
    control:   Installing       : bats-1.8.0-1.el9.noarch                              83/87
    control:   Installing       : mc-1:4.8.26-5.el9.x86_64                             84/87
    control:   Installing       : tree-1.8.0-10.el9.x86_64                             85/87
    control:   Installing       : python3-pip-21.2.3-6.el9.noarch                      86/87
    control:   Installing       : python3-netaddr-0.8.0-5.el9.noarch                   87/87
    control:   Running scriptlet: python3-netaddr-0.8.0-5.el9.noarch                   87/87
    control:   Verifying        : bind-libs-32:9.16.23-11.el9_2.2.x86_64                1/87
    control:   Verifying        : bind-license-32:9.16.23-11.el9_2.2.noarch             2/87
    control:   Verifying        : bind-utils-32:9.16.23-11.el9_2.2.x86_64               3/87
    control:   Verifying        : fstrm-0.6.1-3.el9.x86_64                              4/87
    control:   Verifying        : gpm-libs-1.20.7-29.el9.x86_64                         5/87
    control:   Verifying        : libmaxminddb-1.5.2-3.el9.x86_64                       6/87
    control:   Verifying        : libuv-1:1.42.0-1.el9.x86_64                           7/87
    control:   Verifying        : mc-1:4.8.26-5.el9.x86_64                              8/87
    control:   Verifying        : perl-AutoLoader-5.74-480.el9.noarch                   9/87
    control:   Verifying        : perl-B-1.80-480.el9.x86_64                           10/87
    control:   Verifying        : perl-Carp-1.50-460.el9.noarch                        11/87
    control:   Verifying        : perl-Class-Struct-0.66-480.el9.noarch                12/87
    control:   Verifying        : perl-Data-Dumper-2.174-462.el9.x86_64                13/87
    control:   Verifying        : perl-Digest-1.19-4.el9.noarch                        14/87
    control:   Verifying        : perl-Digest-MD5-2.58-4.el9.x86_64                    15/87
    control:   Verifying        : perl-Encode-4:3.08-462.el9.x86_64                    16/87
    control:   Verifying        : perl-Errno-1.30-480.el9.x86_64                       17/87
    control:   Verifying        : perl-Exporter-5.74-461.el9.noarch                    18/87
    control:   Verifying        : perl-Fcntl-1.13-480.el9.x86_64                       19/87
    control:   Verifying        : perl-File-Basename-2.85-480.el9.noarch               20/87
    control:   Verifying        : perl-File-Path-2.18-4.el9.noarch                     21/87
    control:   Verifying        : perl-File-Temp-1:0.231.100-4.el9.noarch              22/87
    control:   Verifying        : perl-File-stat-1.09-480.el9.noarch                   23/87
    control:   Verifying        : perl-FileHandle-2.03-480.el9.noarch                  24/87
    control:   Verifying        : perl-Getopt-Long-1:2.52-4.el9.noarch                 25/87
    control:   Verifying        : perl-Getopt-Std-1.12-480.el9.noarch                  26/87
    control:   Verifying        : perl-HTTP-Tiny-0.076-460.el9.noarch                  27/87
    control:   Verifying        : perl-IO-1.43-480.el9.x86_64                          28/87
    control:   Verifying        : perl-IO-Socket-IP-0.41-5.el9.noarch                  29/87
    control:   Verifying        : perl-IO-Socket-SSL-2.073-1.el9.noarch                30/87
    control:   Verifying        : perl-IPC-Open3-1.21-480.el9.noarch                   31/87
    control:   Verifying        : perl-MIME-Base64-3.16-4.el9.x86_64                   32/87
    control:   Verifying        : perl-Mozilla-CA-20200520-6.el9.noarch                33/87
    control:   Verifying        : perl-NDBM_File-1.15-480.el9.x86_64                   34/87
    control:   Verifying        : perl-Net-SSLeay-1.92-2.el9.x86_64                    35/87
    control:   Verifying        : perl-POSIX-1.94-480.el9.x86_64                       36/87
    control:   Verifying        : perl-PathTools-3.78-461.el9.x86_64                   37/87
    control:   Verifying        : perl-Pod-Escapes-1:1.07-460.el9.noarch               38/87
    control:   Verifying        : perl-Pod-Perldoc-3.28.01-461.el9.noarch              39/87
    control:   Verifying        : perl-Pod-Simple-1:3.42-4.el9.noarch                  40/87
    control:   Verifying        : perl-Pod-Usage-4:2.01-4.el9.noarch                   41/87
    control:   Verifying        : perl-Scalar-List-Utils-4:1.56-461.el9.x86_64         42/87
    control:   Verifying        : perl-SelectSaver-1.02-480.el9.noarch                 43/87
    control:   Verifying        : perl-Socket-4:2.031-4.el9.x86_64                     44/87
    control:   Verifying        : perl-Storable-1:3.21-460.el9.x86_64                  45/87
    control:   Verifying        : perl-Symbol-1.08-480.el9.noarch                      46/87
    control:   Verifying        : perl-Term-ANSIColor-5.01-461.el9.noarch              47/87
    control:   Verifying        : perl-Term-Cap-1.17-460.el9.noarch                    48/87
    control:   Verifying        : perl-Text-ParseWords-3.30-460.el9.noarch             49/87
    control:   Verifying        : perl-Text-Tabs+Wrap-2013.0523-460.el9.noarch         50/87
    control:   Verifying        : perl-Thread-Queue-3.14-460.el9.noarch                51/87
    control:   Verifying        : perl-Time-Local-2:1.300-7.el9.noarch                 52/87
    control:   Verifying        : perl-URI-5.09-3.el9.noarch                           53/87
    control:   Verifying        : perl-base-2.27-480.el9.noarch                        54/87
    control:   Verifying        : perl-constant-1.33-461.el9.noarch                    55/87
    control:   Verifying        : perl-if-0.60.800-480.el9.noarch                      56/87
    control:   Verifying        : perl-interpreter-4:5.32.1-480.el9.x86_64             57/87
    control:   Verifying        : perl-libnet-3.13-4.el9.noarch                        58/87
    control:   Verifying        : perl-libs-4:5.32.1-480.el9.x86_64                    59/87
    control:   Verifying        : perl-mro-1.23-480.el9.x86_64                         60/87
    control:   Verifying        : perl-overload-1.31-480.el9.noarch                    61/87
    control:   Verifying        : perl-overloading-0.02-480.el9.noarch                 62/87
    control:   Verifying        : perl-parent-1:0.238-460.el9.noarch                   63/87
    control:   Verifying        : perl-podlators-1:4.14-460.el9.noarch                 64/87
    control:   Verifying        : perl-subs-1.03-480.el9.noarch                        65/87
    control:   Verifying        : perl-threads-1:2.25-460.el9.x86_64                   66/87
    control:   Verifying        : perl-threads-shared-1.61-460.el9.x86_64              67/87
    control:   Verifying        : perl-vars-1.05-480.el9.noarch                        68/87
    control:   Verifying        : python3-PyMySQL-0.10.1-6.el9.noarch                  69/87
    control:   Verifying        : python3-netaddr-0.8.0-5.el9.noarch                   70/87
    control:   Verifying        : python3-pip-21.2.3-6.el9.noarch                      71/87
    control:   Verifying        : vim-common-2:8.2.2637-20.el9_1.x86_64                72/87
    control:   Verifying        : vim-enhanced-2:8.2.2637-20.el9_1.x86_64              73/87
    control:   Verifying        : bash-completion-1:2.11-4.el9.noarch                  74/87
    control:   Verifying        : libpkgconf-1.7.3-10.el9.x86_64                       75/87
    control:   Verifying        : pkgconf-1.7.3-10.el9.x86_64                          76/87
    control:   Verifying        : pkgconf-m4-1.7.3-10.el9.noarch                       77/87
    control:   Verifying        : pkgconf-pkg-config-1.7.3-10.el9.x86_64               78/87
    control:   Verifying        : protobuf-c-1.3.3-12.el9.x86_64                       79/87
    control:   Verifying        : python3-cffi-1.14.5-5.el9.x86_64                     80/87
    control:   Verifying        : python3-cryptography-36.0.1-2.el9.x86_64             81/87
    control:   Verifying        : python3-ply-3.11-14.el9.noarch                       82/87
    control:   Verifying        : python3-pycparser-2.20-6.el9.noarch                  83/87
    control:   Verifying        : tree-1.8.0-10.el9.x86_64                             84/87
    control:   Verifying        : vim-filesystem-2:8.2.2637-20.el9_1.noarch            85/87
    control:   Verifying        : bats-1.8.0-1.el9.noarch                              86/87
    control:   Verifying        : parallel-20230822-1.el9.noarch                       87/87
    control:
    control: Installed:
    control:   bash-completion-1:2.11-4.el9.noarch
    control:   bats-1.8.0-1.el9.noarch
    control:   bind-libs-32:9.16.23-11.el9_2.2.x86_64
    control:   bind-license-32:9.16.23-11.el9_2.2.noarch
    control:   bind-utils-32:9.16.23-11.el9_2.2.x86_64
    control:   fstrm-0.6.1-3.el9.x86_64
    control:   gpm-libs-1.20.7-29.el9.x86_64
    control:   libmaxminddb-1.5.2-3.el9.x86_64
    control:   libpkgconf-1.7.3-10.el9.x86_64
    control:   libuv-1:1.42.0-1.el9.x86_64
    control:   mc-1:4.8.26-5.el9.x86_64
    control:   parallel-20230822-1.el9.noarch
    control:   perl-AutoLoader-5.74-480.el9.noarch
    control:   perl-B-1.80-480.el9.x86_64
    control:   perl-Carp-1.50-460.el9.noarch
    control:   perl-Class-Struct-0.66-480.el9.noarch
    control:   perl-Data-Dumper-2.174-462.el9.x86_64
    control:   perl-Digest-1.19-4.el9.noarch
    control:   perl-Digest-MD5-2.58-4.el9.x86_64
    control:   perl-Encode-4:3.08-462.el9.x86_64
    control:   perl-Errno-1.30-480.el9.x86_64
    control:   perl-Exporter-5.74-461.el9.noarch
    control:   perl-Fcntl-1.13-480.el9.x86_64
    control:   perl-File-Basename-2.85-480.el9.noarch
    control:   perl-File-Path-2.18-4.el9.noarch
    control:   perl-File-Temp-1:0.231.100-4.el9.noarch
    control:   perl-File-stat-1.09-480.el9.noarch
    control:   perl-FileHandle-2.03-480.el9.noarch
    control:   perl-Getopt-Long-1:2.52-4.el9.noarch
    control:   perl-Getopt-Std-1.12-480.el9.noarch
    control:   perl-HTTP-Tiny-0.076-460.el9.noarch
    control:   perl-IO-1.43-480.el9.x86_64
    control:   perl-IO-Socket-IP-0.41-5.el9.noarch
    control:   perl-IO-Socket-SSL-2.073-1.el9.noarch
    control:   perl-IPC-Open3-1.21-480.el9.noarch
    control:   perl-MIME-Base64-3.16-4.el9.x86_64
    control:   perl-Mozilla-CA-20200520-6.el9.noarch
    control:   perl-NDBM_File-1.15-480.el9.x86_64
    control:   perl-Net-SSLeay-1.92-2.el9.x86_64
    control:   perl-POSIX-1.94-480.el9.x86_64
    control:   perl-PathTools-3.78-461.el9.x86_64
    control:   perl-Pod-Escapes-1:1.07-460.el9.noarch
    control:   perl-Pod-Perldoc-3.28.01-461.el9.noarch
    control:   perl-Pod-Simple-1:3.42-4.el9.noarch
    control:   perl-Pod-Usage-4:2.01-4.el9.noarch
    control:   perl-Scalar-List-Utils-4:1.56-461.el9.x86_64
    control:   perl-SelectSaver-1.02-480.el9.noarch
    control:   perl-Socket-4:2.031-4.el9.x86_64
    control:   perl-Storable-1:3.21-460.el9.x86_64
    control:   perl-Symbol-1.08-480.el9.noarch
    control:   perl-Term-ANSIColor-5.01-461.el9.noarch
    control:   perl-Term-Cap-1.17-460.el9.noarch
    control:   perl-Text-ParseWords-3.30-460.el9.noarch
    control:   perl-Text-Tabs+Wrap-2013.0523-460.el9.noarch
    control:   perl-Thread-Queue-3.14-460.el9.noarch
    control:   perl-Time-Local-2:1.300-7.el9.noarch
    control:   perl-URI-5.09-3.el9.noarch
    control:   perl-base-2.27-480.el9.noarch
    control:   perl-constant-1.33-461.el9.noarch
    control:   perl-if-0.60.800-480.el9.noarch
    control:   perl-interpreter-4:5.32.1-480.el9.x86_64
    control:   perl-libnet-3.13-4.el9.noarch
    control:   perl-libs-4:5.32.1-480.el9.x86_64
    control:   perl-mro-1.23-480.el9.x86_64
    control:   perl-overload-1.31-480.el9.noarch
    control:   perl-overloading-0.02-480.el9.noarch
    control:   perl-parent-1:0.238-460.el9.noarch
    control:   perl-podlators-1:4.14-460.el9.noarch
    control:   perl-subs-1.03-480.el9.noarch
    control:   perl-threads-1:2.25-460.el9.x86_64
    control:   perl-threads-shared-1.61-460.el9.x86_64
    control:   perl-vars-1.05-480.el9.noarch
    control:   pkgconf-1.7.3-10.el9.x86_64
    control:   pkgconf-m4-1.7.3-10.el9.noarch
    control:   pkgconf-pkg-config-1.7.3-10.el9.x86_64
    control:   protobuf-c-1.3.3-12.el9.x86_64
    control:   python3-PyMySQL-0.10.1-6.el9.noarch
    control:   python3-cffi-1.14.5-5.el9.x86_64
    control:   python3-cryptography-36.0.1-2.el9.x86_64
    control:   python3-netaddr-0.8.0-5.el9.noarch
    control:   python3-pip-21.2.3-6.el9.noarch
    control:   python3-ply-3.11-14.el9.noarch
    control:   python3-pycparser-2.20-6.el9.noarch
    control:   tree-1.8.0-10.el9.x86_64
    control:   vim-common-2:8.2.2637-20.el9_1.x86_64
    control:   vim-enhanced-2:8.2.2637-20.el9_1.x86_64
    control:   vim-filesystem-2:8.2.2637-20.el9_1.noarch
    control:
    control: Complete!
    control: Defaulting to user installation because normal site-packages is not writeable
    control: Collecting ansible
    control:   Downloading ansible-8.5.0-py3-none-any.whl (47.5 MB)
    control: Collecting ansible-core~=2.15.5
    control:   Downloading ansible_core-2.15.5-py3-none-any.whl (2.2 MB)
    control: Requirement already satisfied: PyYAML>=5.1 in /usr/lib64/python3.9/site-packages (from ansible-core~=2.15.5->ansible) (5.4.1)
    control: Collecting importlib-resources<5.1,>=5.0
    control:   Downloading importlib_resources-5.0.7-py3-none-any.whl (24 kB)
    control: Requirement already satisfied: cryptography in /usr/lib64/python3.9/site-packages (from ansible-core~=2.15.5->ansible) (36.0.1)
    control: Collecting resolvelib<1.1.0,>=0.5.3
    control:   Downloading resolvelib-1.0.1-py2.py3-none-any.whl (17 kB)
    control: Collecting packaging
    control:   Downloading packaging-23.2-py3-none-any.whl (53 kB)
    control: Collecting jinja2>=3.0.0
    control:   Downloading Jinja2-3.1.2-py3-none-any.whl (133 kB)
    control: Collecting MarkupSafe>=2.0
    control:   Downloading MarkupSafe-2.1.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
    control: Requirement already satisfied: cffi>=1.12 in /usr/lib64/python3.9/site-packages (from cryptography->ansible-core~=2.15.5->ansible) (1.14.5)
    control: Requirement already satisfied: pycparser in /usr/lib/python3.9/site-packages (from cffi>=1.12->cryptography->ansible-core~=2.15.5->ansible) (2.20)
    control: Requirement already satisfied: ply==3.11 in /usr/lib/python3.9/site-packages (from pycparser->cffi>=1.12->cryptography->ansible-core~=2.15.5->ansible) (3.11)
    control: Installing collected packages: MarkupSafe, resolvelib, packaging, jinja2, importlib-resources, ansible-core, ansible
    control:   WARNING: Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/10151>
    control:   distutils: /home/vagrant/.local/lib/python3.9/site-packages
    control:   sysconfig: /home/vagrant/.local/lib64/python3.9/site-packages
    control:   WARNING: Additional context:
    control:   user = True
    control:   home = None
    control:   root = None
    control:   prefix = None
    control: Successfully installed MarkupSafe-2.1.3 ansible-8.5.0 ansible-core-2.15.5 importlib-resources-5.0.7 jinja2-3.1.2 packaging-23.2 resolvelib-1.0.1
```


#### Vagrant ssh control
```bash
PS C:\Users\JensV\Desktop\Hogent_23-24\Sem1\Infrastructure_Automation\infra-labs-23-24-JensVandeWynckel\vmlab> vagrant ssh control

This system is built by the Bento project by Chef Software
More information can be found at https://github.com/chef/bento
[vagrant@control ~]$
```

## 2.2 Adding a managed node

Next, we are going to set up our first **managed node**, i.e. a host that is managed by Ansible.

Add a new VM named `srv100` (which will become our web server) to the Vagrant environment by editing the `vagrant-hosts.yml` file in the `vmlab` directory.

```yaml
- name: srv100
  ip: 172.16.128.100
  netmask: 255.255.0.0
  box: bento/almalinux-9
```

Your control node should always be the last VM defined in `vagrant-hosts.yml`. The reason will become apparent at the final stage of this assignment.

Check whether the new VM is recognized by Vagrant by running `vagrant status` and look for the host name in the command output. If it is, start the VM with `vagrant up srv100`. Check that you can log in to the VM with `vagrant ssh srv100`.

In order to communicate with managed nodes, you need to provide Ansible with a list of hosts. This list is called an [inventory](https://docs.ansible.com/ansible/latest/user_guide/intro_inventory.html). The inventory can be a simple list of host names, but it can also contain additional information like the host's IP address, the user account to log in with, etc. We will use a simple inventory file that contains only the host name of the VM we just created. Create a file `vmlab/ansible/inventory.yml` with the following contents:
```yaml
---
servers:
  vars:
    ansible_user: vagrant
    ansible_ssh_private_key_file: ../.vagrant/machines/srv100/virtualbox/private_key
    ansible_become: true
  hosts:
    srv100:
      ansible_host: 172.16.128.100
```
```bash session
[vagrant@srv100 ansible]$ touch inventory.yml
[vagrant@srv100 ansible]$ ls
files  group_vars  inventory.yml  site.yml
[vagrant@srv100 ansible]$ vi inventory.yml
```

### 2.2.1 Ansible ping
```bash session
[vagrant@control ansible]$ ping 172.16.128.100
PING 172.16.128.100 (172.16.128.100) 56(84) bytes of data.
64 bytes from 172.16.128.100: icmp_seq=1 ttl=64 time=0.459 ms
64 bytes from 172.16.128.100: icmp_seq=2 ttl=64 time=0.379 ms
64 bytes from 172.16.128.100: icmp_seq=3 ttl=64 time=0.378 ms
64 bytes from 172.16.128.100: icmp_seq=4 ttl=64 time=0.437 ms
^C
--- 172.16.128.100 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3090ms
rtt min/avg/max/mdev = 0.378/0.413/0.459/0.035 ms
```

### 2.2.2 ssh vagrant@srv100
```bash session
[vagrant@control ansible]$ ssh vagrant@172.16.128.100
vagrant@172.16.128.100's password:

This system is built by the Bento project by Chef Software
More information can be found at https://github.com/chef/bento
Last login: Sun Oct 15 18:54:51 2023 from 10.0.2.2
[vagrant@srv100 ~]$
```

### 2.2.3 ssh 
```bash session
[vagrant@control ansible]$ ls -l
total 1
drwxrwxrwx. 1 vagrant vagrant   0 Oct  8 12:33 files
drwxrwxrwx. 1 vagrant vagrant   0 Oct  8 12:33 group_vars
-rwxrwxrwx. 1 vagrant vagrant 213 Oct 15 19:12 inventory.yml
-rwxrwxrwx. 1 vagrant vagrant  17 Oct  8 12:33 site.yml
[vagrant@control ansible]$ ssh -
ssh: Could not resolve hostname -: Name or service not known
[vagrant@control ansible]$ ssh -i ../.vagrant/machines/srv100/virtualbox/private_key vagrant@172.16.128.100
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@         WARNING: UNPROTECTED PRIVATE KEY FILE!          @
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Permissions 0777 for '../.vagrant/machines/srv100/virtualbox/private_key' are too open.
It is required that your private key files are NOT accessible by others.
This private key will be ignored.
Load key "../.vagrant/machines/srv100/virtualbox/private_key": bad permissions
vagrant@172.16.128.100's password:

This system is built by the Bento project by Chef Software
More information can be found at https://github.com/chef/bento
Last login: Sun Oct 15 19:26:30 2023 from 172.16.128.253
[vagrant@srv100 ~]$

```

Change the following in the inventory.yml file
```yaml
    # ansible_ssh_private_key_file: ../.vagrant/machines/srv100/virtualbox/private_key
    # To this:
    ansible_ssh_pass: vagrant
```

### 2.2.4 Ansible ping (Working after inventory.yml changes)
```bash session 
[vagrant@control vagrant]$ ansible -i inventory.yml -m shell -a 'hostname' srv100
[WARNING]: Unable to parse /vagrant/inventory.yml as an inventory source
[WARNING]: No inventory was parsed, only implicit localhost is available
[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'
[WARNING]: Could not match supplied host pattern, ignoring: srv100
[vagrant@control vagrant]$ cd ansible
[vagrant@control ansible]$ ansible -i inventory.yml srv100 -m ping
srv100 | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python3"
    },
    "changed": false,
    "ping": "pong"
}
[vagrant@control ansible]$
```

### 2.2.5 Ansible -i inventory.yml -m setup -a "filter=ansible_distribution*" srv100
```bash session
[vagrant@control ansible]$ ansible -i inventory.yml -m setup -a "filter=ansible_distribution*" srv100
srv100 | SUCCESS => {
    "ansible_facts": {
        "ansible_distribution": "AlmaLinux",
        "ansible_distribution_file_parsed": true,
        "ansible_distribution_file_path": "/etc/redhat-release",
        "ansible_distribution_file_variety": "RedHat",
        "ansible_distribution_major_version": "9",
        "ansible_distribution_release": "Turquoise Kodkod",
        "ansible_distribution_version": "9.2",
        "discovered_interpreter_python": "/usr/bin/python3"
    },
    "changed": false
}
```


### Console Output

#### Ansible -i inventory.yml (FAILS)
```bash session
[vagrant@control ansible]$ ansible -i inventory.yml -m ping srv100
srv100 | UNREACHABLE! => {
    "changed": false,
    "msg": "Failed to connect to the host via ssh: @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\r\n@         WARNING: UNPROTECTED PRIVATE KEY FILE!          @\r\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\r\nPermissions 0777 for '../.vagrant/machines/srv100/virtualbox/private_key' are too open.\r\nIt is required that your private key files are NOT accessible by others.\r\nThis private key will be ignored.\r\nLoad key \"../.vagrant/machines/srv100/virtualbox/private_key\": bad permissions\r\nvagrant@172.16.128.100: Permission denied (publickey,gssapi-keyex,gssapi-with-mic,password).",
    "unreachable": true
}
[vagrant@control ansible]$ ansible -i inventory.yml -m setup srv100
srv100 | UNREACHABLE! => {
    "changed": false,
    "msg": "Failed to connect to the host via ssh: @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\r\n@         WARNING: UNPROTECTED PRIVATE KEY FILE!          @\r\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\r\nPermissions 0777 for '../.vagrant/machines/srv100/virtualbox/private_key' are too open.\r\nIt is required that your private key files are NOT accessible by others.\r\nThis private key will be ignored.\r\nLoad key \"../.vagrant/machines/srv100/virtualbox/private_key\": bad permissions\r\nvagrant@172.16.128.100: Permission denied (publickey,gssapi-keyex,gssapi-with-mic,password).",
    "unreachable": true
}
[vagrant@control ansible]$ ansible -i inventory.yml -m all srv100
srv100 | UNREACHABLE! => {
    "changed": false,
    "msg": "Failed to connect to the host via ssh: @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\r\n@         WARNING: UNPROTECTED PRIVATE KEY FILE!          @\r\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\r\nPermissions 0777 for '../.vagrant/machines/srv100/virtualbox/private_key' are too open.\r\nIt is required that your private key files are NOT accessible by others.\r\nThis private key will be ignored.\r\nLoad key \"../.vagrant/machines/srv100/virtualbox/private_key\": bad permissions\r\nvagrant@172.16.128.100: Permission denied (publickey,gssapi-keyex,gssapi-with-mic,password).",
    "unreachable": true
}
```


#### Install SSHPass
```bash session
[vagrant@control ansible]$ sudo dnf install sshpass
Last metadata expiration check: 2:31:28 ago on Sun 15 Oct 2023 05:05:08 PM UTC.
Dependencies resolved.
===============================================================================
 Package          Architecture    Version             Repository          Size
===============================================================================
Installing:
 sshpass          x86_64          1.09-4.el9          appstream           27 k

Transaction Summary
===============================================================================
Install  1 Package

Total download size: 27 k
Installed size: 47 k
Is this ok [y/N]: y
Downloading Packages:
sshpass-1.09-4.el9.x86_64.rpm                   38 kB/s |  27 kB     00:00
-------------------------------------------------------------------------------
Total                                           17 kB/s |  27 kB     00:01
Running transaction check
Transaction check succeeded.
Running transaction test
Transaction test succeeded.
Running transaction
  Preparing        :                                                       1/1
  Installing       : sshpass-1.09-4.el9.x86_64                             1/1
  Running scriptlet: sshpass-1.09-4.el9.x86_64                             1/1
  Verifying        : sshpass-1.09-4.el9.x86_64                             1/1

Installed:
  sshpass-1.09-4.el9.x86_64

Complete!
[vagrant@control ansible]$
```

#### Vagrant status (after adding srv100 and initializing it)
```bash
PS C:\Users\JensV\Desktop\Hogent_23-24\Sem1\Infrastructure_Automation\infra-labs-23-24-JensVandeWynckel\vmlab> vagrant status
Current machine states:

srv100                    running (virtualbox)
control                   running (virtualbox)

This environment represents multiple VMs. The VMs are all listed
above with their current state. For more information about a specific
VM, run `vagrant status NAME`.
```


#### Vagrant up srv100
```bash
PS C:\Users\JensV\Desktop\Hogent_23-24\Sem1\Infrastructure_Automation\infra-labs-23-24-JensVandeWynckel\vmlab> vagrant up srv100
Bringing machine 'srv100' up with 'virtualbox' provider...
==> srv100: Importing base box 'bento/almalinux-9'...
==> srv100: Matching MAC address for NAT networking...
==> srv100: Checking if box 'bento/almalinux-9' version '202309.08.0' is up to date...
==> srv100: Setting the name of the VM: vmlab_srv100_1697395967122_11420
==> srv100: Fixed port collision for 22 => 2222. Now on port 2201.
==> srv100: Clearing any previously set network interfaces...
==> srv100: Preparing network interfaces based on configuration...
    srv100: Adapter 1: nat
    srv100: Adapter 2: hostonly
==> srv100: Forwarding ports...
    srv100: 22 (guest) => 2201 (host) (adapter 1)
==> srv100: Running 'pre-boot' VM customizations...
==> srv100: Booting VM...
==> srv100: Waiting for machine to boot. This may take a few minutes...
    srv100: SSH address: 127.0.0.1:2201
    srv100: SSH username: vagrant
    srv100: SSH auth method: private key
    srv100:
    srv100: Vagrant insecure key detected. Vagrant will automatically replace
    srv100: this with a newly generated keypair for better security.
    srv100:
    srv100: Inserting generated public key within guest...
    srv100: Removing insecure key from the guest if it's present...
    srv100: Key inserted! Disconnecting and reconnecting using new SSH key...
==> srv100: Machine booted and ready!
==> srv100: Checking for guest additions in VM...
==> srv100: Setting hostname...
==> srv100: Configuring and enabling network interfaces...
==> srv100: Mounting shared folders...
    srv100: /vagrant => C:/Users/JensV/Desktop/Hogent_23-24/Sem1/Infrastructure_Automation/infra-labs-23-24-JensVandeWynckel/vmlab
```	
#### Vagrant ssh srv100
```bash
PS C:\Users\JensV\Desktop\Hogent_23-24\Sem1\Infrastructure_Automation\infra-labs-23-24-JensVandeWynckel\vmlab> vagrant ssh srv100

This system is built by the Bento project by Chef Software
More information can be found at https://github.com/chef/bento
[vagrant@srv100 ~]$
```


## 2.3 Applying a role to a managed node

Next, run the site.yml playbook. (see console output)


### Console Output

#### Ansible-galaxy download
```bash session
[vagrant@control ansible]$ ansible-galaxy install bertvv.rh-base
Starting galaxy role install process
- downloading role 'rh-base', owned by bertvv
- downloading role from https://github.com/bertvv/ansible-role-rh-base/archive/v4.0.4.tar.gz
- extracting bertvv.rh-base to /home/vagrant/.ansible/roles/bertvv.rh-base
- bertvv.rh-base (v4.0.4) was installed successfully
```


#### Run the site.yml playbook
```bash session
[vagrant@control ansible]$ ansible-playbook -i inventory.yml site.yml

PLAY [Configure srv100] **********************************************************

TASK [Gathering Facts] ***********************************************************
ok: [srv100]

TASK [bertvv.rh-base : Load distro specific variables] ***************************
ok: [srv100] => (item=/home/vagrant/.ansible/roles/bertvv.rh-base/vars/RedHat.yml)

TASK [bertvv.rh-base : include_tasks] ********************************************
included: /home/vagrant/.ansible/roles/bertvv.rh-base/tasks/install.yml for srv100

TASK [bertvv.rh-base : Install | Check minimal value of ‘rhbase_repo_installonly_limit’ (>= 2)] ***
ok: [srv100] => {
    "msg": "The value of ‘rhbase_repo_installonly_limit’ should be at least 2, actual value is 3."
}

TASK [bertvv.rh-base : Install | Ensure the machine-ID is available] *************
ok: [srv100]

TASK [bertvv.rh-base : Install | Ensure basic systemd services are running] ******
ok: [srv100] => (item=systemd-journald)
ok: [srv100] => (item=systemd-tmpfiles-setup-dev)
ok: [srv100] => (item=systemd-tmpfiles-setup)

TASK [bertvv.rh-base : Install | Role/Ansible dependencies] **********************
ok: [srv100]

TASK [bertvv.rh-base : Install | Package management configuration (dnf)] *********
changed: [srv100]

TASK [bertvv.rh-base : Install | Ensure specified external repositories are installed] ***
ok: [srv100]

TASK [bertvv.rh-base : Install | Ensure specified repositories are enabled] ******
skipping: [srv100]

TASK [bertvv.rh-base : Install | Ensure specified packages are installed] ********
ok: [srv100]

TASK [bertvv.rh-base : Install | Ensure specified packages are NOT installed] ****
ok: [srv100]

TASK [bertvv.rh-base : Install | Ensure all updates are installed] ***************
skipping: [srv100]

TASK [bertvv.rh-base : include_tasks] ********************************************
included: /home/vagrant/.ansible/roles/bertvv.rh-base/tasks/config.yml for srv100

TASK [bertvv.rh-base : Config | Ensure host name is in the hosts file] ***********
skipping: [srv100]

TASK [bertvv.rh-base : Config | Install dependency for dynamic MotD] *************
skipping: [srv100]

TASK [bertvv.rh-base : Config | Install script generating dynamic MotD] **********
skipping: [srv100]

TASK [bertvv.rh-base : Config | Check if ifup-eth script overrides firewall zone] ***
ok: [srv100]

TASK [bertvv.rh-base : Config | Don’t override firewall zone in ifup-eth script] ***
skipping: [srv100]

TASK [bertvv.rh-base : Config | Check if ifup-post script overrides firewall zone] ***
ok: [srv100]

TASK [bertvv.rh-base : Config | Don’t override firewall zone in ifup-post script] ***
skipping: [srv100]

TASK [bertvv.rh-base : Config | Set the TZ environment variable] *****************
changed: [srv100]

TASK [bertvv.rh-base : Config | Set protocol version for SSH] ********************
changed: [srv100]

TASK [bertvv.rh-base : Config | Set PermitEmptyPasswords] ************************
changed: [srv100]

TASK [bertvv.rh-base : Config | Set IgnoreRhosts] ********************************
changed: [srv100]

TASK [bertvv.rh-base : Config | Set HostbasedAuthentication] *********************
changed: [srv100]

TASK [bertvv.rh-base : include_tasks] ********************************************
included: /home/vagrant/.ansible/roles/bertvv.rh-base/tasks/services.yml for srv100

TASK [bertvv.rh-base : Services | Ensure SSH daemon is running] ******************
ok: [srv100]

TASK [bertvv.rh-base : Services | Ensure `/var/log/journal` exists] **************
changed: [srv100]

TASK [bertvv.rh-base : Services | Ensure specified services are running] *********
skipping: [srv100]

TASK [bertvv.rh-base : Services | Ensure specified services are NOT running] *****
skipping: [srv100]

TASK [bertvv.rh-base : include_tasks] ********************************************
included: /home/vagrant/.ansible/roles/bertvv.rh-base/tasks/security.yml for srv100

TASK [bertvv.rh-base : Security | Make sure SELinux has the desired state (enforcing)] ***
changed: [srv100]

TASK [bertvv.rh-base : Security | Enable SELinux booleans] ***********************
skipping: [srv100]

TASK [bertvv.rh-base : Security | Make sure the firewall is running] *************
changed: [srv100]

TASK [bertvv.rh-base : Security | Make sure basic services can pass through firewall] ***
ok: [srv100] => (item=dhcpv6-client)
ok: [srv100] => (item=ssh)

TASK [bertvv.rh-base : Security | Make sure user specified services can pass through firewall] ***
skipping: [srv100]

TASK [bertvv.rh-base : Security | Make sure user specified ports can pass through firewall] ***
skipping: [srv100]

TASK [bertvv.rh-base : Security | Make sure specified interfaces are added] ******
skipping: [srv100]

TASK [bertvv.rh-base : include_tasks] ********************************************
included: /home/vagrant/.ansible/roles/bertvv.rh-base/tasks/users.yml for srv100

TASK [bertvv.rh-base : Users | Process groups] ***********************************
skipping: [srv100]

TASK [bertvv.rh-base : Users | Add groups] ***************************************
skipping: [srv100]

TASK [bertvv.rh-base : Users | Add users] ****************************************
skipping: [srv100]

TASK [bertvv.rh-base : Users | Set up SSH keys] **********************************
skipping: [srv100]

TASK [bertvv.rh-base : include_tasks] ********************************************
included: /home/vagrant/.ansible/roles/bertvv.rh-base/tasks/admin.yml for srv100

TASK [bertvv.rh-base : Admin | Make sure users from the wheel group can use sudo] ***
changed: [srv100]

TASK [bertvv.rh-base : Admin | Set attributes of sudo configuration file for wheel group] ***
changed: [srv100]

TASK [bertvv.rh-base : Admin | Make sure only these groups can ssh] **************
skipping: [srv100]

TASK [bertvv.rh-base : include_tasks] ********************************************
skipping: [srv100]

RUNNING HANDLER [bertvv.rh-base : restart journald] ******************************
changed: [srv100]

RUNNING HANDLER [bertvv.rh-base : restart sshd] **********************************
changed: [srv100]

PLAY RECAP ***********************************************************************
srv100                     : ok=32   changed=13   unreachable=0    failed=0    skipped=19   rescued=0    ignored=0
```


## 2.4 Web application server 

### Console Output

## 2.5 DNS

### Console Output

## 2.6 DHCP

### Console Output

## 2.7 Router

### Console Output

## 2.8 Integration: a working LAN

### Console Output